# Time Series Data Analysis using Pandas

- Created by Clayton Miller - clayton@nus.edu.sg - miller.clayton@gmail.com

This notebook is an introduction to several key time-series functions available in the Pandas library. 

Several portions of the function explanations are from: https://www.dataquest.io/blog/tutorial-time-series-analysis-with-pandas/

## The Building Data Genome Project

For this set of videos, we will be using the Building Data Genome Project:

https://github.com/buds-lab/the-building-data-genome-project

![alt text](https://raw.githubusercontent.com/buds-lab/the-building-data-genome-project/master/figures/buildingdatagenome1.png)

This project is a set of 500+ time-series meter data from buildings -- check out the website which includes an overview of the data set, sources, and publications that use the data set. 

In this notebook, we will use some of the buildings from that data set to understand why Pandas was designed for time-series data from IoT networks

First, we will load the libraries and mount the drives as usual

In [0]:
import pandas as pd
import os

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
os.chdir("/content/gdrive/My Drive/EDX Data Science for Construction, Architecture and Engineering/Week 3 - Construction - Pandas Fundamentals/meter_data/")

Let's begin this demonstration of time-series data analysis by loading the data from a single building from the `Building Data Genome Project` -- this building has the name `Abilgail`

You can notice that this time there is a column in the data set called `timestamp`. If we load the DataFrame the usual way then this column will just be a string data type:


In [0]:
abigail = pd.read_csv('Office_Abigail.csv', index_col = "timestamp") 

In [0]:
abigail.head()

In [0]:
abigail.info()

In [0]:
abigail.index[0]

## Setting the `timestamp` index to a *datetime* object 

Pandas has several very useful functions that we will cover in this notebook, but first we need to indicate to Pandas that the index is not just a regular string -- it has information related to date and time attributes including whether the day is **Monday** or **Tuesday** or whether that is a weekday or weekend. 

We can tell the `.read_csv()` function that it should **parse** the index and try to convert it into a *datetime* object. Let's see what happens when we do that.

In [0]:
abigail = pd.read_csv('Office_Abigail.csv', index_col = "timestamp", parse_dates=True) 

In [0]:
abigail.head()

In [0]:
abigail.info()

In [0]:
abigail.index[0]

## `Timestamp` object attributes

You will notice that the index is not what's known as a `DateTimeIndex` and each of the index items is a `Timestamp` object. This allows us to as the object things like what day of the week it is and other attributes. 

More info here: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Timestamp.html


In [0]:
abigail.index

In [0]:
abigail.index[0].month

In [0]:
abigail.index[0].dayofweek

In [0]:
abigail.index[0].week

In [0]:
abigail.index[0].is_leap_year

In [0]:
abigail.index[0].hour

# Plotting simple line charts of time-series data

The first thing we can do with the data is to explore it as a line chart. This gives us an understanding of the shape, frequency and potential for outliers. Data exploration is the first step in the analysis process.

In [0]:
abigail.plot()

In [0]:
abigail.plot(marker='.', alpha=0.5, linestyle='None', figsize=(15, 5))

# Resample the data to other frequencies

"It is often useful to resample our time series data to a lower or higher frequency. Resampling to a lower frequency (downsampling) usually involves an aggregation operation — for example, computing monthly sales totals from daily data. The daily OPSD data we’re working with in this tutorial was downsampled from the original hourly time series. Resampling to a higher frequency (upsampling) is less common and often involves interpolation or other data filling method — for example, interpolating hourly weather data to 10 minute intervals for input to a scientific model.

We will focus here on downsampling, exploring how it can help us analyze our OPSD data on various time scales. We use the DataFrame’s `resample()` method, which splits the DatetimeIndex into time bins and groups the data by time bin. The `resample()` method returns a Resampler object, similar to a pandas GroupBy object. We can then apply an aggregation method such as `mean()`, `median()`, `sum()`, etc., to the data group for each time bin." -- from: https://www.dataquest.io/blog/tutorial-time-series-analysis-with-pandas/



In [0]:
abigail.head()

We can first resample to daily data by using the `D` as an attribute in the `.resample()` function to indicate that we want *daily* data aggregation. We use the `.mean()` function to indicate that we want the function to take the average across each day's hourly readings to produce the aggregated value for the day

In [0]:
abigail_daily = abigail.resample("D").mean()

In [0]:
abigail_daily.head()

In [0]:
abigail_daily.plot(figsize=(10,5))

We can do the same to create *monthly* data and other aggregations

In [0]:
abigail_daily.resample("M").mean().plot(figsize=(10,5))

## Frequencies 

The Pandas Library has dozens of **frequencies** that can be used to resample data -- the detailed list is here: https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects


# Truncating Time-Series Data

The truncate function allows us to slice the data into smaller dataframes according to a time range

Pandas DataFrame.truncate() function is used to truncate a Series or DataFrame before and after some index value. This is a useful shorthand for boolean indexing based on index values above or below certain thresholds.

https://www.geeksforgeeks.org/python-pandas-dataframe-truncate/

In [0]:
abigail_daily.info()

In [0]:
abigail_daily_june = abigail_daily.truncate(before = '2015-06-01', after='2015-07-01')

In [0]:
abigail_daily_june.plot(figsize=(10,5))

# Trends Analysis and Rolling Windows

"Time series data often exhibit some slow, gradual variability in addition to higher frequency variability such as seasonality and noise. An easy way to visualize these trends is with rolling means at different time scales.

A rolling mean tends to smooth a time series by averaging out variations at frequencies much higher than the window size and averaging out any seasonality on a time scale equal to the window size. This allows lower-frequency variations in the data to be explored. Since our electricity consumption time series has weekly and yearly seasonality, let’s look at rolling means on those two time scales." - https://www.dataquest.io/blog/tutorial-time-series-analysis-with-pandas/

In [0]:
abigail.rolling(window=500, center=True, min_periods=500).mean().plot()

# Analysis of a large number of buildings at once

Let's look at a larger group of buildings to understand how these functions can enhanced the analysis of a large number of buildings

In [0]:
os.listdir()

Using that list, let's take all the buildings that have a time range of 2015

In [0]:
list_of_buildings = ['UnivClass_Andy.csv',
'Office_Abbey.csv',
'Office_Alannah.csv',
'PrimClass_Angel.csv',
'Office_Penny.csv',
'Office_Pam.csv',
'UnivClass_Craig.csv',
'UnivLab_Allison.csv',
'Office_Amelia.csv',
'Office_Aubrey.csv',
'Office_Cecelia.csv',
'UnivClass_Conor.csv',
'Office_Autumn.csv',
'Office_Abigail.csv',
'Office_Amelie.csv',
'UnivClass_Alfredo.csv',
'Office_Phebian.csv',
'UnivLab_Adrian.csv',
'UnivDorm_Curtis.csv',
'UnivLab_Angie.csv',
'UnivClass_Amya.csv',
'UnivDorm_Cian.csv',
'UnivClass_Ciara.csv',
'UnivLab_Audra.csv',
'UnivLab_Ciel.csv',
'UnivLab_Cesar.csv']

In [0]:
all_data_list = []
for buildingname in list_of_buildings:
  df = pd.read_csv(buildingname, index_col = "timestamp", parse_dates=True) 
  df = df.resample("H").mean()
  all_data_list.append(df)
all_data = pd.concat(all_data_list, axis=1)

In [0]:
all_data.info()

In [0]:
all_data.plot(figsize=(20,30), subplots=True);

In [0]:
all_data.resample("D").mean().plot(figsize=(20,30), subplots=True);

In [0]:
all_data.resample("D").mean().truncate(after='2015-06-01').plot(figsize=(20,30), subplots=True);

## Filter columns by query

Let's just look at the *office* buildings by filtering the columns based on name

In [0]:
all_data.columns[all_data.columns.str.contains("Office")]

In [0]:
all_data[all_data.columns[all_data.columns.str.contains("Office")]].info()

In [0]:
all_data[all_data.columns[all_data.columns.str.contains("Office")]].plot(figsize=(15,15), subplots=True);